<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-кода" data-toc-modified-id="Подготовка-кода-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка кода</a></span><ul class="toc-item"><li><span><a href="#Загрузка-библиотек" data-toc-modified-id="Загрузка-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка библиотек</a></span></li></ul></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Теоретическое-обоснование" data-toc-modified-id="Теоретическое-обоснование-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Теоретическое обоснование</a></span><ul class="toc-item"><li><span><a href="#Решение-задачи-для-матрицы-признков" data-toc-modified-id="Решение-задачи-для-матрицы-признков-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Решение задачи для матрицы признков</a></span></li><li><span><a href="#Решение-задачи-для-произведения-матриц" data-toc-modified-id="Решение-задачи-для-произведения-матриц-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Решение задачи для произведения матриц</a></span></li><li><span><a href="#Сравнение-вариантов-умножения-матриц" data-toc-modified-id="Сравнение-вариантов-умножения-матриц-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Сравнение вариантов умножения матриц</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Вспомогательные-материалы" data-toc-modified-id="Вспомогательные-материалы-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Вспомогательные материалы</a></span></li><li><span><a href="#Преобразование" data-toc-modified-id="Преобразование-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Преобразование</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Наша задача разработать метод защиты данных клиентов, такой, чтобы:
- по преобразованным данным было сложно восстановить персональную информацию. 
- при преобразовании качесво моделей машинного обучения не ухудшилось.

Требуется обосновать корректность работы метода преобразования. Проверку работы медота сделаем для алгоритма линейной регресии. 

Подбор наилучшей модели в границы проекта не входит. 

## Подготовка кода

### Загрузка библиотек

In [1]:
from os import path

import pandas as pd
import numpy as np

## Загрузка данных

In [8]:
data_remote_url = 'https://code.s3.yandex.net/datasets/insurance.csv'
data_local_path = path.join('.', 'data', 'insurance.csv')

if path.isfile(data_local_path):
    data = pd.read_csv(data_local_path)
else:
    data = pd.read_csv(data_remote_url)
    if path.exists(path.dirname(data_local_path)):
        data.to_csv(data_local_path)
        
data.shape

(5000, 5)

## Умножение матриц

In [3]:
X = np.arange(25).reshape(5,5)
X.shape

(5, 5)

In [6]:
X.T @ X == X @ X.T

array([[False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False,  True, False],
       [False, False,  True, False, False],
       [False, False, False, False, False]])

In [7]:
X

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [8]:
X.T

array([[ 0,  5, 10, 15, 20],
       [ 1,  6, 11, 16, 21],
       [ 2,  7, 12, 17, 22],
       [ 3,  8, 13, 18, 23],
       [ 4,  9, 14, 19, 24]])

In [9]:
X @ X.T

array([[  30,   80,  130,  180,  230],
       [  80,  255,  430,  605,  780],
       [ 130,  430,  730, 1030, 1330],
       [ 180,  605, 1030, 1455, 1880],
       [ 230,  780, 1330, 1880, 2430]])

In [10]:
X.T @ X

array([[ 750,  800,  850,  900,  950],
       [ 800,  855,  910,  965, 1020],
       [ 850,  910,  970, 1030, 1090],
       [ 900,  965, 1030, 1095, 1160],
       [ 950, 1020, 1090, 1160, 1230]])

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w \mathit{MSE}(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** ...

**Обоснование:** ...

### Теоретическое обоснование

В данной задаче мы рассматриваем алгоритм линейной регресии, который предполагает, что между независимыми переменными и зависимой переменной существует линейная зависимость. 

Введем обозначения:
- $k, k \in \mathbb{N}, k > 0$ — количество независимых переменных,
- $m, m \in \mathbb{N}, m > k$ — количество наблюдений,
- $n = k + 1$,
- $X, X \in \mathbb{R}^{m \times n}$ — матрица независимых переменных, у которой первый столбец содержит 1 для всех наблюдений,
- $y, y \in \mathbb{R}^m$ — вектор значений зависимой переменной,
- $w, w \in \mathbb{R}^n$ — вектор весов линейной регресии, нулевой элемент равен сдвигу.

Тогда, принимая, что между $X$ и $y$ есть линейная зависимость, определяемая вектором $w$, верно равенство:

$y = Xw$ 

#### Решение задачи для матрицы признков

Из этого равенства мы можем расчитать значения вектора $w$:
1. умножим слева обе части уравнения на $X^T$, чтобы получить квадратную матрицу $X^T X$ размером $m \times n$
    - $X^T y = X^T X w$ 
2. если $(X^T X)$ обратима, то есть существует такая матрица $(X^T X)^{-1}$, что $(X^T X)^{-1} (X^T X) = E$, то умножим слева обе части уравнения на $(X^T X)^{-1}$ и получим
    - $(X^T X)^{-1} X^T y = (X^T X)^{-1} (X^T X) w = w$.
    
Мы получили формулу обучения $w = (X^T X)^{-1} X^T y$

#### Решение задачи для произведения матриц

У нас стоит задача защитить значения $X$ от передачи сервису обучения и прогнозирования. 

Рассмотрим подход, когда мы умножаем значения $X$ на матрицу $P, P \in \mathbb{R}^{k \times l}$ перед отравкой в сервис обучения и прогнозирования. Поскольку умножение матриц не коммутативно, у нас может быть два варианта:

$y = (P X) w$

$y = (X P) w$

Мы знаем, что размер $X$ — $m \times n$, размер $w$ — $1 \times n$, размер $y$ — $1 \times m$, тогда размер $(P X)$ и $(X P)$ должен быть $m \times n$ и 
- в первом случае размер $P$ — $m \times m$
- во втором случае размер $P$ — $n \times n$

Обозначим произведение матриц $X$ и $P$ как $Q$.

Тогда имеем уравнение

$y = Qw$,

которое решаем аналогичным для $y = Xw$ и получаем 

$w = (Q^T Q)^{-1} Q^T y$, 

где $Q = PX, P \in \mathbb{R}^{m \times m}$ или $Q = XP, P \in \mathbb{R}^{n \times n}$

#### Сравнение вариантов умножения матриц

Таким образом мы показали, что если мы умножим матрицу $X$ на матрицу $P$, и если произведение матриц будет обратимым, то мы можем решить уравнение для произведения матриц тем же способом, что и для матрицы признаков. 

Так мы доказали, что можем использовать произведение матриц для обучения и предсказаний с тем же результатом, что и для матрицы признаков.  

Мы можем умножить матрицу признаков $X$ как слева, так и справа. Оценим ресурсы.

Умножение матриц рамеров $m \times k$ на $k \times m$ стоит $ 2mnk $ flops. Тогда
- умножение слева $P X$ будет стоить $2 m m n = 2m^2n $
- умножение справа $X P$ будет стоить $2 m n n = 2mn^2 $

По условию $m \ge n$, тогда $2m^2n \ge 2mn^2$ и второй вариант — умножение справа — предпочтительный.

#### Вывод

Мы показали, что если между признаками и целевым признаком есть линейная зависимость, и обратимы матрицы $(X^T X)$ и матрицы $(Q^T Q)$, где $Q = X P$, и $X, Q \in \mathbb{R}^{m \times n}, P \in \mathbb{R}^{n \times n}$, то решение для вычисления коэффициентов линейной регрессии идентично для $X$ и $Q$, а значит мы можем использовать произведение матриц для обучения и прогнозирования, не раскрывая данных клиентов. 

### Вспомогательные материалы

Подставим значение $w$ в формулу линейной регресси:

$ a = X (X^T X)^{-1} X^T y $

При условии, что $X^T$ — обратимая матрица, из определения обратимой матрицы следует:

$ (X^T)^{-1} X^T = I $

Из определения идентичной матрицы, следует, что умножение матрицы на идентичную дает матрицу:

$ A I = I A = A $

Тогда мы можем записать 

$ a = X (X^T X)^{-1} X^T y = I X (X^T X)^{-1} X^T y = (X^T)^{-1} X^T X (X^T X)^{-1} X^T y $

Из свойств произведения матриц мы знаем, что произведение ассоциативно, то есть 

$ (AB)C = A(BC) = ABC $

Тогда 

$ a = (X^T)^{-1} (X^T X) (X^T X)^{-1} X^T y = (X^T)^{-1} ((X^T X) (X^T X)^{-1}) X^T y $

По определению обратной матрицы 

$ ((X^T X) (X^T X)^{-1}) = I $

Тогда 

$ a = (X^T)^{-1} (I) X^T y = ((X^T)^{-1} X^T) y $

$ (X^T)^{-1} X^T) = I $

Тогда 

$ a = I y = y $

### Преобразование

Покажем, что для $ X \in \mathbb{R}^{m \times n}, Z \in \mathbb{R}^{m \times m}$ и $Z^{-1}Z = E$, $a, y \in \mathbb{R}^m$ и $w \in \mathbb{R}^n$, таком, что 

$ w(X) = (X^T X)^{-1} X^T y $

$ a = X w(X) = Z X w(Z X) = y $

Нам понадобятся следующие свойства матриц:
- $ A A^{-1} = A^{-1} A = E $
- $ (A B)^{-1} = B^{-1} A^{-1} $
- $(A B)^T = B^T A^T$

## Алгоритм преобразования

**Алгоритм**

...

**Обоснование**

...

## Проверка алгоритма

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования